In [2]:
!pip install selenium
!pip install bs4

  Using cached selenium-4.32.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.32.0-py3-none-any.whl (9.4 MB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv

# 1. Setup headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

all_products = []

# 2. Loop through multiple Flipkart search result pages
for page in range(1, 30): 
    url = f"https://www.flipkart.com/search?q=laptop&page={page}"
    driver.get(url)
    time.sleep(3)  # Allow page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all("div", {"data-id": True})

    for item in products:
        # Basic Details
        name_tag = item.find("div", class_="KzDlHZ")
        specs_tag = item.find("div", class_="_6NESgJ")
        rating_tag = item.find("div", class_="XQDdHH") or item.find("span", class_="Y1HW00")
        price_tag = item.find("div", class_="hl05eU") or item.find("span", class_="cN1yY0")
        link_tag = item.find("a", href=True)

        name = name_tag.get_text().strip() if name_tag else "Not found"
        specs = specs_tag.get_text().strip() if specs_tag else "Not found"
        rating = rating_tag.get_text().strip() if rating_tag else "Not found"
        price = price_tag.get_text().strip() if price_tag else "Not found"
        product_url = "https://www.flipkart.com" + link_tag["href"] if link_tag else "Not found"

        # Extra Details from Product Page
        battery = weight = webcam = display = "Not found"

        if product_url != "Not found":
            try:
                driver.get(product_url)
                time.sleep(2)
                detail_soup = BeautifulSoup(driver.page_source, "html.parser")
                table_cells = detail_soup.find_all("td")

                for i in range(len(table_cells)):
                    label = table_cells[i].get_text().lower()
                    value = table_cells[i + 1].get_text() if i + 1 < len(table_cells) else ""

                    if "battery" in label or "battery backup" in label:
                        battery = value
                    elif "weight" in label:
                        weight = value
                    elif "web camera" in label or "webcam" in label:
                        webcam = value
                    elif "display size" in label or "screen size" in label:
                        display = value

            except Exception as e:
                print(f"⚠️ Failed to extract details from {product_url}")

        # Save product data
        all_products.append([
            name, specs, rating, price, product_url,
            battery, weight, webcam, display
        ])

    print(f"✅ Page {page} scraped — Total products so far: {len(all_products)}")

# 3. Close browser
driver.quit()

# 4. Save data to CSV
with open("flipkart_laptop_final.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Product Name", "Specifications", "Rating", "Price", "Product URL",
        "Battery Life", "Weight", "Webcam", "Display Size"
    ])
    writer.writerows(all_products)

print("✅ Saved data to 'flipkart_laptop_final.csv'")


✅ Page 1 scraped — Total products so far: 24
✅ Page 2 scraped — Total products so far: 48
✅ Page 3 scraped — Total products so far: 72
✅ Page 4 scraped — Total products so far: 96
✅ Page 5 scraped — Total products so far: 120
✅ Page 6 scraped — Total products so far: 144
✅ Page 7 scraped — Total products so far: 168
✅ Page 8 scraped — Total products so far: 192
✅ Page 9 scraped — Total products so far: 216
✅ Page 10 scraped — Total products so far: 240
✅ Page 11 scraped — Total products so far: 264
✅ Page 12 scraped — Total products so far: 288
✅ Page 13 scraped — Total products so far: 312
✅ Page 14 scraped — Total products so far: 336
✅ Page 15 scraped — Total products so far: 360
✅ Page 16 scraped — Total products so far: 384
✅ Page 17 scraped — Total products so far: 408
✅ Page 18 scraped — Total products so far: 432
✅ Page 19 scraped — Total products so far: 456
✅ Page 20 scraped — Total products so far: 480
✅ Page 21 scraped — Total products so far: 504
✅ Page 22 scraped — Total 

In [5]:
!pip install pandas




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
# Load the CSV file into a DataFrame
import pandas as pd
df = pd.read_csv("flipkart_laptop_final.csv")

# Show the first 5 rows to confirm
df.head()

,Product Name,Specifications,Rating,Price,Product URL,Battery Life,Weight,Webcam,Display Size
0,HP Backlit Intel Core i5 12th Gen 1235U - (8 G...,Intel Core i5 Processor (12th Gen)8 GB DDR4 RA...,4.3,"₹44,990₹70,06435% off",https://www.flipkart.com/hp-backlit-intel-core...,Warranty Summary,1.69 Kg,HP True Vision 720p HD camera,39.62 cm (15.6 Inch)
1,"ASUS Vivobook 16X for Creator, Intel Core i5 1...",Intel Core i5 Processor (12th Gen)16 GB DDR4 R...,4.2,"₹66,490₹84,99021% off",https://www.flipkart.com/asus-vivobook-16x-cre...,Not found,1.80 Kg,720p HD camera With privacy shutter,40.64 cm (16 Inch)
2,HP Intel Core i5 12th Gen 1235U - (16 GB/512 G...,Intel Core i5 Processor (12th Gen)16 GB DDR4 R...,4.3,"₹49,990₹62,52620% off",https://www.flipkart.com/hp-intel-core-i5-12th...,Not found,1.69 Kg,HP True Vision 720p HD camera with temporal no...,39.62 cm (15.6 inch)
3,ASUS Vivobook Go 15 AMD Ryzen 3 Quad Core 7320...,AMD Ryzen 3 Quad Core Processor8 GB LPDDR5 RAM...,4.2,"₹29,990₹44,99033% off",https://www.flipkart.com/asus-vivobook-go-15-a...,3-cell Li-ion,1.63 Kg,720p HD camera With privacy shutter,39.62 cm (15.6 Inch)
4,ASUS Vivobook 15 Intel Core i3 13th Gen 1315U ...,Intel Core i3 Processor (13th Gen)16 GB DDR4 R...,4.3,"₹37,990₹56,99033% off",https://www.flipkart.com/asus-vivobook-15-inte...,3-cell Li-ion,1.7 Kg,720p HD camera With privacy shutter,39.62 cm (15.6 Inch)


In [8]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Product Name    696 non-null    object
 1   Specifications  696 non-null    object
 2   Rating          696 non-null    object
 3   Price           696 non-null    object
 4   Product URL     696 non-null    object
 5   Battery Life    696 non-null    object
 6   Weight          696 non-null    object
 7   Webcam          696 non-null    object
 8   Display Size    696 non-null    object
dtypes: object(9)
memory usage: 49.1+ KB


In [9]:
df.isnull().sum()

Product Name      0
Specifications    0
Rating            0
Price             0
Product URL       0
Battery Life      0
Weight            0
Webcam            0
Display Size      0
dtype: int64

In [10]:
df.duplicated().sum()

0

In [11]:
df.dtypes

Product Name      object
Specifications    object
Rating            object
Price             object
Product URL       object
Battery Life      object
Weight            object
Webcam            object
Display Size      object
dtype: object

In [12]:
df.nunique()

Product Name      407
Specifications    376
Rating             21
Price             407
Product URL       696
Battery Life       76
Weight            116
Webcam            105
Display Size       26
dtype: int64

In [13]:
df.columns

Index(['Product Name', 'Specifications', 'Rating', 'Price', 'Product URL',
       'Battery Life', 'Weight', 'Webcam', 'Display Size'],
      dtype='object')

In [14]:
# =========================
# Price Cleaning
# =========================
df['Price'] = (
    df['Price'].astype(str)
    .str.extract(r'₹([\d,]+)')[0]
    .str.replace(',', '', regex=True)
    .astype(float)
)
df = df.dropna(subset=['Price'])
df['Price'] = df['Price'].astype(int)


In [15]:
df['Price'].describe()

count       696.000000
mean      54090.268678
std       25794.861414
min       10999.000000
25%       37341.000000
50%       49990.000000
75%       69990.000000
max      232490.000000
Name: Price, dtype: float64

In [16]:
# =========================
# Remove 'Not found'  prices
# =========================
df = df[~(df['Price'] == 'Not found')]


In [18]:
# =========================
# Clean Product Name
# =========================
import re
def extract_laptop_name(text):
    return re.split(r'Intel|AMD|\(|-', text)[0].strip()

df['Clean Name'] = df['Product Name'].apply(extract_laptop_name)
df['Product Name'] = df['Clean Name']
df.drop(columns=['Clean Name'], inplace=True)


In [22]:
# # =========================
# # Brand Extraction (First word of name)
# # =========================
df['Brand'] = df['Product Name'].apply(lambda x: x.split()[0])


In [23]:
# # =========================
# # Fix spacing between letters and digits in 'Specifications'
# # =========================
df["Specifications"] = df["Specifications"].astype(str).apply(
    lambda x: re.sub(r"(?<=[a-zA-Z])(?=\d)", " ", x)
)


In [24]:
# =========================
# Extract core features from 'Specifications'
# =========================
def extract_feature(pattern, text):
    match = re.search(pattern, text)
    return match.group(0).strip() if match else "Not found"

df["Processor"] = df["Specifications"].apply(lambda x: extract_feature(r"(Intel|AMD|Celeron)[^,\n]*Processor\s*\([^)]*\)?", x))
df["RAM"] = df["Specifications"].apply(lambda x: extract_feature(r"\d+\s*GB", x))
df["DDR"] = df["Specifications"].apply(lambda x: extract_feature(r"(LP)?DDR\d?", x))
df["OS"] = df["Specifications"].apply(lambda x: extract_feature(r"Windows\s+11\s*(Home)?", x))
df["Storage"] = df["Specifications"].apply(lambda x: extract_feature(r"\d+\s*(GB|TB)\s*SSD", x))
df["Display"] = df["Specifications"].apply(lambda x: extract_feature(r"\d{2}\.\d{2}\s*cm\s*\([^)]+\)", x))
df["Warranty"] = df["Specifications"].apply(lambda x: extract_feature(r"\d+\s*Year\s+(Onsite|Carry-in)?\s*Warranty", x))


In [25]:
# =========================
# Final formatted 'text' column
# =========================


def format_product(row):
    return (
        f"Product: {row['Product Name']}, Brand: {row['Brand']}, Rating: {row['Rating']}, Price: ₹{row['Price']}, "
        f"Processor: {row['Processor']}, RAM: {row['RAM']} GB, DDR: {row['DDR']}, OS: {row['OS']}, "
        f"Storage: {row['Storage']}, "
        f"Weight: {row['Weight']} kg, "
        f"Warranty: {row['Warranty']}"
    )
df["text"] = df.apply(format_product, axis=1)

In [26]:
# =========================
# Export final clean dataset
# =========================
df.to_csv("scraped_products.csv", index=False)
print("✅ Cleaned data saved to flipkart_laptop_cleaned.csv")


✅ Cleaned data saved to flipkart_laptop_cleaned.csv
